In [1]:
import os
os.environ['HF_HOME'] = '/data/loriss21dm/cache'
from datasets import load_dataset

In [2]:
l = list(range(0,100))
l[(len(l)//10)-1::len(l)//10]

[9, 19, 29, 39, 49, 59, 69, 79, 89, 99]

In [3]:
len(l)//10

10

In [4]:
dataset_name = "loris3/stratified_equitoken_10m_curriculum"

In [5]:
dataset = load_dataset(dataset_name)

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'stage'],
        num_rows: 100000
    })
    validation: Dataset({
        features: ['text', 'source', 'stage'],
        num_rows: 53457
    })
})

In [4]:
df = dataset["train"].to_pandas()

In [5]:
df["source"].unique()

array(['aochildes.txt', 'children_stories.txt', 'switchboard.txt',
       'bnc_spoken.txt', 'switchboard.txt, bnc_spoken.txt', 'qed.txt',
       'wikipedia.txt'], dtype=object)

In [6]:
import json
import pandas as pd
from huggingface_hub import hf_hub_download
info = None
with open(hf_hub_download(repo_id=dataset_name, filename="info.json", repo_type="dataset")) as f:
    info = json.load(f)

info["curriculum"]

{'C1: Child Directed Speech': ['aochildes.txt'],
 "C2: Children's Books": ['children_stories.txt',
  'cbt_test.txt',
  'cbt_train.txt',
  'cbt_valid.txt'],
 'C3: Dialogue': ['switchboard.txt', 'bnc_spoken.txt', 'open_subtitles.txt'],
 'C4: Educational': ['qed.txt', 'simple_wiki.txt'],
 'C5: Written English': ['wikipedia.txt', 'gutenberg.txt']}

In [7]:
df["word_count"] = df["text"].apply(lambda d: len(d.split()))

In [8]:
df.groupby(by="stage").describe()

word_count                                          \
                               count   mean  std    min    25%    50%    75%   
stage                                                                          
C1: Child Directed Speech    20000.0  100.0  0.0  100.0  100.0  100.0  100.0   
C2: Children's Books         20000.0  100.0  0.0  100.0  100.0  100.0  100.0   
C3: Dialogue                 20000.0  100.0  0.0  100.0  100.0  100.0  100.0   
C4: Educational              20000.0  100.0  0.0  100.0  100.0  100.0  100.0   
C5: Written English          20000.0  100.0  0.0  100.0  100.0  100.0  100.0   

                                  
                             max  
stage                             
C1: Child Directed Speech  100.0  
C2: Children's Books       100.0  
C3: Dialogue               100.0  
C4: Educational            100.0  
C5: Written English        100.0

In [9]:
df.groupby(by="stage").describe()

word_count                                          \
                               count   mean  std    min    25%    50%    75%   
stage                                                                          
C1: Child Directed Speech    20000.0  100.0  0.0  100.0  100.0  100.0  100.0   
C2: Children's Books         20000.0  100.0  0.0  100.0  100.0  100.0  100.0   
C3: Dialogue                 20000.0  100.0  0.0  100.0  100.0  100.0  100.0   
C4: Educational              20000.0  100.0  0.0  100.0  100.0  100.0  100.0   
C5: Written English          20000.0  100.0  0.0  100.0  100.0  100.0  100.0   

                                  
                             max  
stage                             
C1: Child Directed Speech  100.0  
C2: Children's Books       100.0  
C3: Dialogue               100.0  
C4: Educational            100.0  
C5: Written English        100.0

In [10]:
counts = df.groupby(by="stage")["stage"].count()
counts.name = "Documents"

In [11]:
counts_proportion = df.groupby(by="stage")["stage"].count() / df.groupby(by="stage")["stage"].count().sum()
counts_proportion = counts_proportion.map(lambda n: '{:,.2%}'.format(n))
counts_proportion.name = ""

In [12]:
words = df.groupby(by="stage")["word_count"].sum()
words.name = "Words"

In [13]:
words_proportion = df.groupby(by="stage")["word_count"].sum() / df["word_count"].sum()
words_proportion = words_proportion.map(lambda n: '{:,.2%}'.format(n))
words_proportion.name = ""

In [14]:
stats = pd.concat([words,words_proportion, counts, counts_proportion], axis=1)
stats.index.name = "Stage"
stats

,Words,,Documents,
Stage,,,,
C1: Child Directed Speech,2000000,20.00%,20000,20.00%
C2: Children's Books,2000000,20.00%,20000,20.00%
C3: Dialogue,2000000,20.00%,20000,20.00%
C4: Educational,2000000,20.00%,20000,20.00%
C5: Written English,2000000,20.00%,20000,20.00%
